In [1]:
import pandas as pd
original_data = pd.read_csv('jobs_skills.csv') #original data loaded
import re
import string


In [2]:
def removePunctuation(x):
    # Lowercasing all words
    x = x.lower()
    # Removing non ASCII chars
    x = re.sub(r'[^\x00-\x7f]',r' ',x)
    # Removing (replacing with empty spaces actually) all the punctuations
    return re.sub("["+string.punctuation+"]", " ", x)

In [3]:
original_data["skills"] = original_data["skills"].map(removePunctuation)
original_data["title"] = original_data["title"].map(removePunctuation)

In [5]:
original_data.to_csv( "skills_preprocessing.csv", index=True)
dpre= pd.read_csv("skills_preprocessing.csv")

In [6]:
!pip install pymongo

# import data into mongodb 

In [6]:
#indeces.create()
from pymongo import MongoClient
import pymongo

client = MongoClient()

In [7]:
## this error because database is already created

DB_NAME = 'job_skill'
db = client[DB_NAME]
elmenus_collection = db['data']

records_ = dpre.to_dict(orient = 'records')
result = elmenus_collection.insert_many(records_ )


BulkWriteError: batch op errors occurred

In [8]:
client=MongoClient()
db=client.job_skill
collection=db.data
end_data=collection.find()

In [6]:
print(end_data)

In [9]:
collection.create_index([("title",1),("skills",1)])


'title_1_skills_1'

# full text search in MongoDB

In [10]:
import time
start_time = time.time()
search=collection.find_one( { "title": "property consultant" } ) 
print(search)
end_time = time.time()
search_time_MongoDB= end_time - start_time
print("\nsearch time using MongoDB: ",search_time_MongoDB)

{'_id': '5da1795ae43fd1a7ab67ddd9', 'Unnamed: 0': 75174, 'title': 'property consultant', 'jobFunction': "['Sales/Retail']", 'industry': "['Real Estate/Property Management']", 'skills': '  customer support    sales    commercial projects    property    real estate    customer service    sales skills  '}

search time using MongoDB:  0.03300142288208008


# partial matching in MongoDB

In [11]:

start_time = time.time()
search=collection.find_one( { "title": "admin " } ) 
print(search)
end_time = time.time()
search_time_MongoDB= end_time - start_time
print("\nsearch time using MongoDB: ",search_time_MongoDB)

None

search time using MongoDB:  0.0030007362365722656


# Indexing in Elastic Search

In [12]:
!pip install elasticsearch

In [13]:
 # Importing the required libraries
from elasticsearch import Elasticsearch
import pandas as pd
from elasticsearch.helpers import bulk

In [55]:
!pip install mongo-connector
!pip install elastic2-doc-manager

In [14]:
dictionary_skills = dpre.to_dict('records')

In [15]:
 # Creating the client that connects to Elasticsearch sever
ES = Elasticsearch([{'host':'localhost','port':9200}])

In [16]:
 # Delete the index if exists
if ES.indices.exists("example_index"):
    ES.indices.delete(index="example_index") 

In [17]:
 # Prepare the index settings
settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            'title': {'type': 'text'},
            'jobFunction' : {'type': 'text'},
            'industry' : {'type': 'text'},
            'skills':{'type': 'text'}
        }
    }
}

In [18]:
# Create the index with the previous settings
res = ES.indices.create(index="example_index", ignore=400, body=settings)

In [19]:
#Bulk for Indexing large datasets without putting them into memory
res = bulk(ES, dictionary_skills, index='example_index', raise_on_error=True)

# full text search using Elastic Search

In [20]:
# Searching
start_time = time.time()
search=ES.search(index='example_index',body = {'query':{'match':{'title':'admin assistant'}}})
end_time = time.time()
print(search)
search_time_es = end_time - start_time
print ( '\nSearch Time: ', search_time_es ) ## return time in secondes


{'took': 512, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3441, 'relation': 'eq'}, 'max_score': 9.28945, 'hits': [{'_index': 'example_index', '_type': '_doc', '_id': '5da1244fe43fd1a7ab672f70', '_score': 9.28945, '_source': {'Unnamed: 0': 30524, 'title': 'admin assistant', 'jobFunction': "['Administration']", 'industry': "['Marketing and Advertising']", 'skills': '  administration    office management    microsoft office  '}}, {'_index': 'example_index', '_type': '_doc', '_id': '5da12476e43fd1a7ab672fc1', '_score': 9.28945, '_source': {'Unnamed: 0': 30608, 'title': 'admin assistant', 'jobFunction': "['Administration']", 'industry': "['Engineering Services']", 'skills': '  administration    office management    secretary    admin  '}}, {'_index': 'example_index', '_type': '_doc', '_id': '5da12479e43fd1a7ab672fc8', '_score': 9.28945, '_source': {'Unnamed: 0': 30621, 'title': 'admin assistant', 'jobFunction': "['Admi

# partial search using Elastic Search

In [21]:
# Searching
start_time = time.time()
search=ES.search(index='example_index',body = {'query':{'match':{'title':'admin'}}})
end_time = time.time()
print(search)
search_time_es = end_time - start_time
print ( '\nSearch Time: ', search_time_es )

{'took': 7, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1823, 'relation': 'eq'}, 'max_score': 5.5880904, 'hits': [{'_index': 'example_index', '_type': '_doc', '_id': '5da12863e43fd1a7ab673809', '_score': 5.5880904, '_source': {'Unnamed: 0': 32728, 'title': 'admin', 'jobFunction': "['Administration']", 'industry': "['Energy and Utilities', 'Construction - Residential & Commercial/Office', 'Engineering Services']", 'skills': '  office management    administration    admin    secretary    microsoft office  '}}, {'_index': 'example_index', '_type': '_doc', '_id': '5da0ef38e43fd1a7ab66bbb6', '_score': 5.5880904, '_source': {'Unnamed: 0': 901, 'title': 'admin', 'jobFunction': "['Administration']", 'industry': "['Energy and Utilities', 'Construction - Residential & Commercial/Office', 'Engineering Services']", 'skills': '  office management    administration    admin    secretary    microsoft office  '}}, {'_index': 'exa